In [ ]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, pair_confusion_matrix, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, validation_curve
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

plt.style.use('dark_background')

%run DataCleaningFunctions.ipynb

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/cdavidshaffer/CPSC4970-AI/master/data/penguins.csv")
data.info()
data.head(10)

<h2><u>Data Cleaning</u></h2>

In [ ]:
# check text column unique values
textual_columns = ['species', 'island', 'sex']
print_unique_values(textual_columns, data)

['Adelie' 'Chinstrap' 'Gentoo']
['Torgersen' 'Biscoe' 'Dream']
['MALE' 'FEMALE' '?' '_']


In [ ]:
# Replace invalid text entries
replace_characters = ['?', '_']

for char in replace_characters:
    replace_data_values(data, 'sex', char, None)
data.info()

In [ ]:
# convert text columns to string type
for i in textual_columns:
    data[i] = data[i].astype('string')
data.info()

In [ ]:
# clean numerical columns from non-numerical values
numerical_columns = ['culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g']
check_numerical_column_errors(data, numerical_columns)
data.info()

In [ ]:
# Drop all rows with missing values
data = data.dropna()
data.info()

<h3><u>Data Preparation</u></h3>

In [ ]:
X = data.drop("species", axis=1)
y = data["species"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
display(X_train.shape) 
display(X_test.shape)
display(y_train.shape)
display(y_test.shape)

(266, 6)

(67, 6)

(266,)

(67,)

In [ ]:
X_train_numerical = X_train[numerical_columns]
X_test_numerical = X_test[numerical_columns]
display(X_train_numerical.describe())
display(X_test_numerical.describe())

,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g
count,266.000000,266.000000,266.000000,266.000000
mean,44.497744,17.134962,201.808271,4253.289474
std,5.365930,2.020036,13.755167,793.770550
min,34.000000,13.100000,172.000000,2700.000000
25%,39.725000,15.300000,190.250000,3650.000000
50%,45.350000,17.450000,198.000000,4100.000000
75%,49.075000,18.700000,214.000000,4850.000000
max,59.600000,21.500000,230.000000,6300.000000


,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g
count,67.000000,67.000000,67.000000,67.000000
mean,41.988060,17.283582,197.626866,4023.507463
std,5.451731,1.762109,14.638923,829.976732
min,32.100000,13.700000,174.000000,2850.000000
25%,37.700000,16.250000,187.500000,3375.000000
50%,41.300000,17.200000,193.000000,3800.000000
75%,45.750000,18.500000,207.500000,4612.500000
max,54.300000,21.100000,231.000000,5950.000000


In [ ]:
# separate numerical and non-numerical data
X_train_nonnumerical = X_train.drop(columns=numerical_columns)
X_test_nonnumerical = X_test.drop(columns=numerical_columns)
display(X_train_nonnumerical.describe())
display(X_test_nonnumerical.describe())

X_train_numerical = X_train[numerical_columns]
X_test_numerical = X_test[numerical_columns]
display(X_train_numerical.describe())
display(X_test_numerical.describe())

<h3><u>Feature Engineering (SOM)</u></h3>

In [ ]:
som = SOM(m=5, n=5, dim=X_train_numerical.shape[1], random_state=42)
som.fit(X_train_numerical.values)
som_train_predictions = som.predict(X_train_numerical.values)
som_test_predictions = som.predict(X_test_numerical.values)
print(som_train_predictions)
print(som_test_predictions)

[14 16  3 17 23  9 19  8 18 23  1  6  4  5 18 15  6  4  8 23  8 13 10  4
  4 24 23 24 10 14 24 12  7 15  9 14  0 13  6  4  4  5  6 21 13 17  6 18
  6 16  2 22 10 23 16  8  0 17  8 21 22 18  0 21 18  0  6 21 21  4  7  8
  6  5  9 10 14 15  8  0  4  9 19  1 13 23 19  9 24 19 16 15 15 23  4  8
  8  5  9 14  7  4 15  5 17  4 16 14  4 13 15 16  0 14 18 14  4 18 20 14
  5  9 23 13 18 23  6 24 13 21 23 16  5 14 24  9 17 14  4 16 17 24 10 18
  3 23 21 23  4  5  9  6  3 14  3  6 10  4  3  1 14  7 21 23 18 12  6 24
  9  8  5  8 16 13 16  3  7  3  0 21 22  8 16 14  9  4 21 21  4  5 10 17
  0  2 17 13  8 19 19 14 19  0 12 14 19  4  4 18  5 24  7  4  6  2 13  6
 17  6 17  5 14  8 19 20  3 13 16 16  2  0  4 10  4  9 21 22  6  5  9  0
 17 20 21  8  3 22 11  4 19  3  4  8  5 15 23 22 19 13 18  3 17 22 23 15
  8 15]
[15  8 24  6 24 14  4  0  1 23 14 19 13  5 19 10 21  5 17 19 12  6  4 24
  0 21  9 14  6  4 21  4 19 22  4 13 17 23  5 21 16  5 10 23 16 22  3 18
 23  8  6  4 14  0 23  3 15 19 23 12  4  4 

In [ ]:
X_train_nonnumerical_som = pd.DataFrame(np.append(X_train_nonnumerical, som_train_predictions.reshape(-1, 1), axis=1), columns=X_train_nonnumerical.columns.tolist() + ['SOM Category'])
X_test_nonnumerical_som = pd.DataFrame(np.append(X_test_nonnumerical, som_test_predictions.reshape(-1, 1), axis=1), columns=X_train_nonnumerical.columns.tolist() + ['SOM Category'])


display(X_train_nonnumerical_som.shape)
display(X_test_nonnumerical_som.shape)
display(X_train_nonnumerical_som.columns.tolist())
display(X_test_nonnumerical_som.columns.tolist())
display(X_train_nonnumerical_som.head(5))
display(X_test_nonnumerical_som.head(5))
preprocessor = make_column_transformer(
    (OneHotEncoder(), X_train_nonnumerical_som.columns.tolist()),
    remainder='passthrough'
)
X_train_processed = preprocessor.fit_transform(X_train_nonnumerical_som)
X_test_processed = preprocessor.transform(X_test_nonnumerical_som)
display(X_train_processed.shape)
display(X_test_processed.shape)


(266, 3)

(67, 3)

['island', 'sex', 'SOM Category']

['island', 'sex', 'SOM Category']

,island,sex,SOM Category
0,Biscoe,FEMALE,14
1,Dream,FEMALE,16
2,Biscoe,MALE,3
3,Biscoe,FEMALE,17
4,Biscoe,MALE,23


,island,sex,SOM Category
0,Dream,FEMALE,15
1,Biscoe,FEMALE,8
2,Torgersen,MALE,24
3,Dream,FEMALE,6
4,Biscoe,MALE,24


(266, 30)

(67, 30)